***GENERATED CODE FOR regrmodeeel PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'index_stringindexer', 'threshold': 22672.742, 'transformation_label': 'Binarizer'}], 'feature': 'index_stringindexer', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '135553', 'mean': '22116.04', 'stddev': '27900.55', 'min': '0.0', 'max': '86409.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'index_stringindexer'}, {'feature_label': 'index_stringindexer', 'threshold': 22672.742, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('index_stringindexer')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Project Number_stringindexer', 'threshold': 21735.312, 'transformation_label': 'Binarizer'}], 'feature': 'Project Number_stringindexer', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '135553', 'mean': '21177.26', 'stddev': '27361.07', 'min': '0.0', 'max': '84720.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Project Number_stringinde...'}, {'feature_label': 'Project Number_stringindexer', 'threshold': 21735.312, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop(
            'Project Number_stringindexer')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run regrmodeeelHooks.ipynb
try:
	#sourcePreExecutionHook()

	filenmae = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-af4c1599-ddc1-4e4e-b813-768a88b2251e-c000.csv', 'filename': '1708314561FileNmae.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/MavenMarket/FileNmae.csv', 'viewFileName': 'FileNmae.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regrmodeeelHooks.ipynb
try:
	#transformationPreExecutionHook()

	regrmodeeelautofe = TransformationMain.run(filenmae,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Zip Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "11941.05", "stddev": "774.15", "min": "501", "max": "15086", "missing": "0"}, "updatedLabel": "Zip Code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Inverter Quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "7.67", "stddev": "10.46", "min": "1", "max": "800", "missing": "0"}, "updatedLabel": "Total Inverter Quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total PV Module Quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "77.74", "stddev": "417.6", "min": "1", "max": "14800", "missing": "0"}, "updatedLabel": "Total PV Module Quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Nameplate kW DC", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "24.32", "stddev": "135.07", "min": "0.0", "max": "5799.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total Nameplate kW DC"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Expected KWh Annual Production", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "30450.46", "stddev": "176375.93", "min": "164", "max": "6807591", "missing": "0"}, "updatedLabel": "Expected KWh Annual Produ..."}, {"transformationsData": [{"feature_label": "index_stringindexer", "threshold": 22672.742, "transformation_label": "Binarizer"}], "feature": "index_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "22116.04", "stddev": "27900.55", "min": "0.0", "max": "86409.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "index_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Reporting Period_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "126.19", "stddev": "240.58", "min": "0.0", "max": "1687.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Reporting Period_stringin..."}, {"transformationsData": [{"feature_label": "Project Number_stringindexer", "threshold": 21735.312, "transformation_label": "Binarizer"}], "feature": "Project Number_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "21177.26", "stddev": "27361.07", "min": "0.0", "max": "84720.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Project Number_stringinde..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "City_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "93.09", "stddev": "206.24", "min": "0.0", "max": "1799.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "City_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "County_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "4.48", "stddev": "8.98", "min": "0.0", "max": "61.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "County_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "State_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "State_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sector_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Sector_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Program Type_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Program Type_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Solicitation_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "1.01", "stddev": "1.94", "min": "0.0", "max": "44.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Solicitation_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Electric Utility_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "1.26", "stddev": "1.68", "min": "0.0", "max": "6.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Electric Utility_stringin..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Purchase Type_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.34", "stddev": "0.62", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Purchase Type_stringindex..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Date Application Received_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "278.32", "stddev": "486.09", "min": "0.0", "max": "3268.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Date Application Received..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Date Completed_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "270.05", "stddev": "466.22", "min": "0.0", "max": "2858.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Date Completed_stringinde..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Project Status_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.47", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Project Status_stringinde..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Contractor_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "18.7", "stddev": "59.12", "min": "0.0", "max": "821.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Contractor_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Primary Inverter Manufacturer_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "2.6", "stddev": "7.64", "min": "0.0", "max": "227.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Primary Inverter Manufact..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Primary Inverter Model Number_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "24.0", "stddev": "73.71", "min": "0.0", "max": "1008.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Primary Inverter Model Nu..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Primary PV Module Manufacturer_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "5.67", "stddev": "18.85", "min": "0.0", "max": "420.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Primary PV Module Manufac..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PV Module Model Number_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "57.12", "stddev": "182.82", "min": "0.0", "max": "2016.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PV Module Model Number_st..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Project Cost_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "5548.85", "stddev": "10465.71", "min": "0.0", "max": "40626.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Project Cost_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Incentive_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "1161.11", "stddev": "2763.79", "min": "0.0", "max": "16812.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Incentive_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Remote Net Metering_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.25", "stddev": "0.43", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Remote Net Metering_strin..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Affordable Solar_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.5", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Affordable Solar_stringin..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Community Distributed Generation_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.5", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Community Distributed Gen..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Green Jobs Green New York Participant_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "135553", "mean": "0.0", "stddev": "0.02", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Green Jobs Green New York..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Location 1_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "135553", "mean": "143.55", "stddev": "298.84", "min": "0.0", "max": "2515.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Location 1_stringindexer"}]}))

	#transformationPostExecutionHook(regrmodeeelautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regrmodeeelHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(regrmodeeelautofe, ["Zip Code", "Total Inverter Quantity", "Total PV Module Quantity", "Total Nameplate kW DC", "Expected KWh Annual Production", "Reporting Period_stringindexer", "City_stringindexer", "County_stringindexer", "State_stringindexer", "Sector_stringindexer", "Program Type_stringindexer", "Solicitation_stringindexer", "Electric Utility_stringindexer", "Purchase Type_stringindexer", "Date Application Received_stringindexer", "Date Completed_stringindexer", "Project Status_stringindexer", "Contractor_stringindexer", "Primary Inverter Manufacturer_stringindexer", "Primary Inverter Model Number_stringindexer", "Primary PV Module Manufacturer_stringindexer", "PV Module Model Number_stringindexer", "Incentive_stringindexer", "Remote Net Metering_stringindexer", "Affordable Solar_stringindexer", "Community Distributed Generation_stringindexer", "Green Jobs Green New York Participant_stringindexer", "Location 1_stringindexer", "index_stringindexer_binarizer", "Project Number_stringindexer_binarizer"], "Project Cost_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

